In [ ]:
import math
import glob
import pickle
import os
import numpy as np
import cv2

from tqdm.notebook import tqdm
from collections import OrderedDict
import matplotlib.pyplot as plt
import matplotlib
from pylab import cm

import constants as c

from bmog import BMOG
from roi_extractor import regions_of_interest

from libs.efficientnet.efficientnet.tfkeras import preprocess_input
import efn

In [ ]:
# LOAD MODEL AND WEIGHTS
model = efn.build_model(phi=-5, dropout=0.15)
model_checkpoints = "../output/efn/-5-dropout_0.15-default/checkpoints/*.hdf5"
model.load_weights(max(glob.iglob(model_checkpoints), key=os.path.getctime))

img_shape = model.input_shape[1:3]

def inference(image):
    img = cv2.resize(image, img_shape) # resize incoming image to models input shape
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # cv2 reads images in BGR, model is trained on RGB
    img = np.expand_dims(img, axis=0) # model expects batches of images
    img = preprocess_input(img) # apply preprocessing function (torch-normalization)
    
    return model.predict(img)[0][0]

In [ ]:
def imread_scaled(image_path, scale):
    image = cv2.imread(image_path)
    w,h,c = image.shape
    w,h = int(w*scale), int(h*scale)
    return cv2.resize(image, (h,w))

def save_roi_pred(image_dir, bmog_threshold_l, bmog_postprocessing_size, dilation, min_size, init_frames, image_scale=1.0):
    found_rois_at_config = {}

    total_amount_iters = len(glob.glob(image_dir+"/*/*.jpg"))
    with tqdm(total=total_amount_iters) as pbar:
        for sequence in glob.glob(image_dir+"/*"):
            seq_name = os.path.basename(sequence)
            rois_for_sequence = {}

            bgs = BMOG(threshold_l=bmog_threshold_l, postprocessing_size=bmog_postprocessing_size)

            # initialize distributions
            for image_path in sorted(glob.glob(f'{sequence}/*.jpg'))[:init_frames]: 
                bgs.apply(imread_scaled(image_path, image_scale)) 
                pbar.update()

            # do roi extraction
            for image_path in sorted(glob.glob(f'{sequence}/*.jpg'))[init_frames:]:
                image_name = os.path.basename(image_path)[:-4] 
                image = imread_scaled(image_path, image_scale)

                #image = cv2.medianBlur(image, blur_kernel)
                fg_mask = bgs.apply(image)

                if dilation:
                    fg_mask = cv2.dilate(fg_mask,(5,5),
                                        iterations=dilation)

                rois = regions_of_interest(fg_mask,
                                        min_size=min_size,
                                        max_size=c.max_size*image_scale,
                                        aspect_ratio=c.aspect_ratio)

                rois_for_sequence[image_name] = []
                for roi in rois:
                    x,y,x2,y2 = roi
                    roi_crop = image[y:y2, x:x2]
                    p = inference(roi_crop)
                    rois_for_sequence[image_name].append([x,y,x2,y2,p])
                pbar.update()
                #print(rois_for_sequence[image_name])
            found_rois_at_config[seq_name] = rois_for_sequence
    
    
    file_name = f'inference_scale-{image_scale}'
    with open(f'{c.output_dir_inference}/{file_name}.pickle', 'wb') as handle:
        pickle.dump(found_rois_at_config, handle, protocol=4)

In [ ]:
import xmltodict

def bbs_from_xml(xml):
    x = xmltodict.parse(xml)
    bbs = []
    
    if 'object' in x['annotation']:  # check for bbs in annoataion file
        for obj in x['annotation']['object']:
            if type(obj) == str: # only one obj in annotation
                obj = x['annotation']['object']
                bbs.append(extract_information(obj))
                break
            bbs.append(extract_information(obj))
    return bbs

def extract_information(xml_object):
    bndbox = xml_object['bndbox']
    bnd_box = [
            int(bndbox['xmin']),
            int(bndbox['ymin']),
            int(bndbox['xmax']),
            int(bndbox['ymax']),
            xml_object['name'],
            ]
    return bnd_box
    
def get_annotations_from_dir(annotation_dir):
    annotations = {}
    for file in glob.glob(f'{annotation_dir}/*.xml'):
        bbs = bbs_from_xml(open(file, 'rb'))
        file_base_name = (os.path.basename(file)[:-4])
        annotations[file_base_name] = bbs
    return annotations


gt_bbs = {}
for sequence in glob.glob(f'{c.test_sequences_dir}/*'):
    seq_name = os.path.basename(sequence)
    annotation_dir = os.path.join(c.test_annotation_dir, seq_name.partition('-')[0])
    
    gt_bbs[seq_name] = get_annotations_from_dir(annotation_dir)

with open(f'{c.output_dir_inference}/gt_bbs_test.pickle', 'wb') as handle:
    pickle.dump(gt_bbs, handle, protocol=4)

In [ ]:
save_roi_pred(c.test_sequences_dir, bmog_threshold_l=20, bmog_postprocessing_size=15, dilation=15, min_size=15, init_frames=c.frames_for_bgs_init, image_scale=1.0) # default (no scale..)

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou


def weird_division(n, d):
    return n / d if d else 0


def statistics_for_rois_pred(_found, _gt, iou_min, scale=1.0, threshold=0.5):
    iou_min_day, iou_night = iou_min

    results = {}

    # get every sequence of GTs
    for seq_name in _gt.keys():
        if 'day' in seq_name:
            iou_for_seq = iou_min_day
        else:
            iou_for_seq = iou_night
            
        
        not_even_found = 0
        should_have_been_found = 0
        FOUND_IN_SEQ = 0
        if seq_name in _found.keys():
            not_even_found = 0
            should_have_been_found = 0
            FOUND_IN_SEQ = 0
            
            tps = 0
            fns = 0
            fps = 0
            tn = 0

            # check every prediction for each frame
            for file_name, gts in _gt[seq_name].items():
                if file_name in _found[seq_name]:
                    for gt in gts:
                        gt = np.array(np.array(gt[0:4])*scale, dtype=int)

                        should_have_been_found += 1
                        found_intersecting_bb = False

                        # compare annotated roi with found rois, and stop when match is found
                        for found in _found[seq_name][file_name]:
                            iou = bb_intersection_over_union(gt, found)
                            if iou > iou_for_seq:
                                found_intersecting_bb = True
                                
                                if found[4] > threshold:
                                    tps += 1
                                else:
                                    fns += 1

                        if not found_intersecting_bb:
                            fps += 1
                            not_even_found += 1

                    FOUND_IN_SEQ += len(_found[seq_name][file_name])
        results[seq_name] = not_even_found, should_have_been_found, FOUND_IN_SEQ, tps, fns, fps
        #results_scale[seq_name] = 20, 100, 120
    return results

In [ ]:
gt_bbs = {}
pred_bbs = {}

with open(f'{c.output_dir_inference}/gt_bbs_test.pickle', 'rb') as handle:
    gt_bbs = pickle.load(handle)
    
with open(f'{c.output_dir_inference}/inference_scale-1.0.pickle', 'rb') as handle:
    pred_bbs = pickle.load(handle)

iou_min = [0.4, 0.3]
statistics = statistics_for_rois_pred(pred_bbs, gt_bbs, iou_min, threshold=0.7)

not_found = []
total = []
seqs = []
been_found = []

true_positives = []
false_negatives = []
false_positives = []


n_seqs = len(gt_bbs.keys())

for seq_name, res in sorted(statistics.items()):
    not_found.append(res[0])
    total.append(res[1])
    been_found.append(res[2])

    true_positives.append(res[3])
    false_positives.append(res[5])
    false_negatives.append(res[4])
    
    # beautify names
    seq_name = seq_name.replace("_", " ").replace("-", " - ")[3:]
    seq_name = seq_name.replace("day", "Tag").replace("dawn", "Nacht").replace("night", "Nacht").replace("animals", "Tiere")
    seq_name = seq_name.replace("orig", "Original").replace("fog", "Nebel").replace("rain", "Regen")
    
    seqs.append(seq_name)

    
true_negatives = np.subtract(been_found, 
                             np.array([true_positives, false_positives, false_negatives, not_found]).sum(axis=0))

In [ ]:
pos = np.arange(len(seqs))
bar_width = 0.9

#plt.bar(pos, been_found, color='C7', width=1, alpha=0.3, label="Gefunden")
plt.bar(pos, total, color='C7', width=1, alpha=0.5, label="Gesucht")


cmap = cm.get_cmap('viridis', n_seqs*3)
greens = [matplotlib.colors.rgb2hex(cmap(idx)[:3]) for idx in [22, 29]]

reds = [matplotlib.colors.rgb2hex(cmap(idx)[:3]) for idx in [4, 10, 18]]

my_cmap = []
handles = []
for i in range(3):
    for j in range(3):
        idx = (i*9)+2*j
        color = matplotlib.colors.rgb2hex(cmap(idx)[:3])
        my_cmap.append(color)
        handles.append(plt.Rectangle((0,0),1,1, color=color))

# tps, tns, fps, fns, nf
my_colors = ["#86B300",
             "#F29718",
             "#EA6C6D",
             "#46BA94",
             "#3199E1"]


plt.bar(pos, not_found, color=my_colors[4], label="Nicht gefunden")

bottom = not_found
plt.bar(pos, false_negatives, bottom=bottom, color=my_colors[3], label="False Negatives")

bottom = np.add(bottom, false_negatives)
plt.bar(pos, true_positives, bottom=bottom, label="True Positives", color=my_colors[0])

bottom = np.add(bottom, true_positives)
plt.bar(pos, false_positives, bottom=bottom, label="False Positives", color=my_colors[2])

bottom = np.add(bottom, false_positives)
plt.bar(pos, true_negatives, bottom=bottom, label="True Negatives", color=my_colors[1])


#plt.plot(pos, been_found, label="Anzahl vorhergesagter ROIs")


#plt.title(f"Inferenz - Confusion Bars")
plt.xticks([], [])
plt.xlim(-0.5, n_seqs-0.5)
#plt.ylim(0, 180)

"""
for p in ax.patches[n_seqs:]:
    h = p.get_height()
    x = p.get_x()+p.get_width()/2.
    y = p.get_y()
    if h != 0:
        ax.annotate("%g" % p.get_height(), xy=(x,y), xytext=(0,4), rotation=0, 
                   textcoords="offset points", ha="center", va="bottom")
"""
plt.legend(
            loc='upper center', bbox_to_anchor=(0.5, -0.02),
            fancybox=True, shadow=True, ncol=3)

plt.savefig(f"../visuals/inference-cmbars.png", dpi=300, bbox_inches='tight')

In [ ]:
false_negatives_true = np.add(false_negatives, not_found)

recall = np.divide(true_positives, np.add(true_positives,false_positives))
precision = np.divide(true_positives, np.add(true_positives,false_negatives_true))
f1 = np.divide(np.multiply(np.multiply(precision, recall), 2), np.add(precision, recall))

_r = sum(true_positives) / (sum(true_positives)+sum(false_positives))
_p = sum(true_positives) / (sum(true_positives)+sum(false_negatives_true))
_f1 = 2 * _r * _p / (_r + _p)

total_r = round(_r*100,1)
total_p = round(_p*100,1)
total_f1 = round(_f1*100,1)

In [ ]:
#plt.title("F1 Score")

fig, axs = plt.subplots(2,2)
fig.subplots_adjust(hspace=0.3, wspace=0.3)


pos = np.arange(len(gt_bbs.keys()))

cmap = cm.get_cmap('viridis', n_seqs*3)
my_cmap = []
handles = []
for i in range(3):
    for j in range(1,4):
        idx = (i*n_seqs)+2*j
        color = matplotlib.colors.rgb2hex(cmap(idx)[:3])
        my_cmap.append(color)
        
        handles.append(plt.Rectangle((0,0),1,1, color=color))

#handles.append(plt.Line2D([0], [0], color="k", lw=4))
#handles.append(plt.Line2D([0], [0], color='k', ls="--"))
     
avg = sum(not_found) / sum(total)
avg_perc = round(np.average(not_found)/np.average(total)*100,1)
avg_bar = avg * np.average(total)
axs[0,0].bar(pos, total, color='k', width=1, alpha=0.2)
axs[0,0].plot([-0.5, 8.5], [avg_bar, avg_bar], "--", color='k', lw=1)
axs[0,0].bar(pos, not_found, color=my_cmap)
axs[0,0].set_title(f"Nicht Gefunden ({avg_perc}%)")


axs[0,1].bar(pos, f1, color=my_cmap)
axs[0,1].plot([-0.5, 8.5], [_f1, _f1], "--", color='k', lw=1)
axs[0,1].set_title(f"F1 Score ({total_f1}%)")
axs[0,1].set_ylim(0, 1)


axs[1,1].bar(pos, recall, color=my_cmap)
axs[1,1].plot([-0.5, 8.5], [_r, _r], "--", color='k', lw=1)
axs[1,1].set_title(f"Recall ({total_r}%)")
axs[1,1].set_ylim(0, 1)


axs[1,0].bar(pos, precision, color=my_cmap)
axs[1,0].plot([-0.5, 8.5], [_p, _p], "--", color='k', lw=1)
axs[1,0].set_title(f"Precision ({total_p}%)")
axs[1,0].set_ylim(0, 1)


for ax in fig.get_axes():
    #ax.label_outer()
    ax.set_xlim(-0.5, 8.5)
    ax.set_xticklabels([], [])
    
    """
    for p in ax.patches[n_seqs:]:
        h = p.get_height()
        x = p.get_x()+p.get_width()/2.
        if h != 0:
            ax.annotate("%g" % p.get_height(), xy=(x,h), xytext=(0,4), rotation=90, 
                       textcoords="offset points", ha="center", va="bottom")
    """
import itertools
def flip(items, ncol):
    return itertools.chain(*[items[i::ncol] for i in range(ncol)])

#plt.xlim(0.4,9-0.5)

plt.legend(handles, seqs,
            loc='upper center', bbox_to_anchor=(-0.2, -0.05),
            fancybox=True, shadow=True, ncol=3)

plt.savefig(f"../visuals/inference-metrics.png", dpi=300, bbox_inches='tight')

In [ ]:
print("Sequenz \t\t\t\t ROIs \t gesucht \t nicht gefunden \t TPs \t FPs \t FNs \t TNs")

for i in range(n_seqs):
    print(seqs[i][:14],
         f"\t\t\t\t {been_found[i]} \t {total[i]} \t\t {not_found[i]} \t\t\t {true_positives[i]} \t {false_positives[i]} \t {false_negatives[i]} \t {true_negatives[i]}")
    

## Test different scales for `min-obj-size` determination

In [ ]:
postprocessors = np.flip([13,13,11,9,7,7,5,5,5]) # must be odd, thus cannot simply scale them

for scale, post in zip(np.around(np.linspace(0.1, 1, 9, endpoint=False), 1), postprocessors):
        save_roi_pred(c.test_sequences_dir, bmog_threshold_l=20,
                         bmog_postprocessing_size=post,
                         dilation=post,
                         min_size=post,
                         init_frames=c.frames_for_bgs_init,
                         image_scale=scale)
# and move to 'output/bmog_scaling' + number them!

In [ ]:
with open('../output/inference/gt_bbs_test.pickle', 'rb') as handle:
    gt_bbs = pickle.load(handle)

fig, axs = plt.subplots(2,1)
fig.subplots_adjust(hspace=0.3, wspace=0.3)

ax_i = 0
for iou_min in [[0.4, 0.3], [0.3, 0.2]]:

    statics = {}
    for rois_config_path in glob.glob(f"../output/inference/inference_scale-*.pickle"):
        config_name = os.path.basename(rois_config_path)

        # last param (-1), cut '.pickle', get number
        scale = float(config_name.split('_')[-1][:-7].split('-')[-1])



        with open(rois_config_path, 'rb') as handle:
            found_rois = pickle.load(handle)
            statics[config_name] = statistics_for_rois_pred(found_rois, gt_bbs, iou_min=iou_min, scale=scale)

    SCALES = len(statics.keys())
    SEQS = len(gt_bbs.keys())
    i = 1

    _found_amount = []
    _not_found = []

    for config_name in sorted(statics.keys())[:]:
        not_found = []
        total = []
        seqs = []
        been_found = []

        for seq_name, res in sorted(statics[config_name].items()):
            if res[1]:
                not_found.append(res[0])
                total.append(res[1])
                been_found.append(res[-1])
                seqs.append(seq_name)

        pos = np.arange(len(seqs)*(i-1), len(seqs)*i)
        bar_width = 0.9

        label = config_name.replace("_", ", ").replace("-", ": ")[10:][:-7].title()

        axs[ax_i].bar(pos, total, color='k', width=1, alpha=0.2, label="Gesuchte Traktoren")
        axs[ax_i].bar(pos, not_found, width=bar_width, alpha=0.9)#, label=label)
        #plt.plot([pos[0],pos[-1]], [sum(not_found)/9, sum(not_found)/9], "--", color=f'C{i-1}', label="Insgesamt nicht gefunden")
        #plt.plot(pos, np.divide(been_found,1), ":", label="Vorhergesagte ROIs")

        i +=1

        _found_amount.append(sum(been_found))
        _not_found.append(sum(not_found))


    not_found_percent = np.multiply(np.divide(_not_found, sum(total)), 100)

    pos = np.linspace(0, (SEQS-1)*SCALES, SCALES)+4
    axs[ax_i].plot(pos, not_found_percent, "o", color="k", label="% nicht gefunden")

    for i in range(SCALES):
        axs[ax_i].annotate(f"{round(not_found_percent[i], 1)}", # this is the text
                 (pos[i], not_found_percent[i]), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(5,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center


    #plt.plot(np.arange(i-1)*SEQS+2.5, np.divide(_found_amount, 10), ":", marker='o', markersize=3, color='k', label="Insgesamt vorhergesagte ROIs / 10", lw=1)

    #plt.title(f"BGS - Bildskalierung")
    axs[ax_i].set_xticks(np.linspace(4, 84, 10))
    axs[ax_i].set_xticklabels(np.linspace(10, 100, 10, dtype=int))
    axs[ax_i].set_xlabel("Skalierung")
    
    axs[ax_i].set_title(f"IoU - Tag {iou_min[0]} & Nacht {iou_min[1]}")
    
    axs[ax_i].set_ylim(0, 150)
    axs[ax_i].set_xlim(-1, SCALES*SEQS)

    #axs[ax_i].set_grid(axis='y', alpha=0.2, linestyle="-")

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))

    plt.legend(by_label.values(), by_label.keys(),
                loc='upper center', bbox_to_anchor=(0.5, -0.35),
                fancybox=True, shadow=True, ncol=4)

    leg = ax.get_legend()
    #leg.legendHandles[0].set_color('C7')
    #leg.legendHandles[1].set_color('C7')
    
    ax_i +=1

for ax in fig.get_axes():
    ax.label_outer()

plt.savefig(f"../visuals/bgs-scaling.png", dpi=300, bbox_inches='tight')

In [ ]:
with open('../output/inference/gt_bbs_test.pickle', 'rb') as handle:
    gt_bbs = pickle.load(handle)


fig, axs = plt.subplots(2,1)
ax_i = 0
for iou_min in [[0.4, 0.3], [0.3, 0.2]]:

    statics = {}
    for rois_config_path in glob.glob(f"../output/inference/inference_scale-*.pickle"):
        config_name = os.path.basename(rois_config_path)

        # last param (-1), cut '.pickle', get number
        scale = float(config_name.split('_')[-1][:-7].split('-')[-1])



        with open(rois_config_path, 'rb') as handle:
            found_rois = pickle.load(handle)
            statics[config_name] = statistics_for_rois_pred(found_rois, gt_bbs, iou_min=iou_min, scale=scale)

    SCALES = len(statics.keys())
    SEQS = len(gt_bbs.keys())
    i = 1

    _found_amount = []
    _not_found = []

    for config_name in sorted(statics.keys())[:]:
        not_found = []
        total = []
        seqs = []
        been_found = []

        for seq_name, res in sorted(statics[config_name].items()):
            if res[1]:
                not_found.append(res[0])
                total.append(res[1])
                been_found.append(res[-1])
                seqs.append(seq_name)

        pos = np.arange(len(seqs)*(i-1), len(seqs)*i)
        bar_width = 0.9

        label = config_name.replace("_", ", ").replace("-", ": ")[10:][:-7].title()

        axs[ax_i].bar(pos, total, color='k', width=1, alpha=0.2, label="Gesuchte Traktoren")
        axs[ax_i].bar(pos, not_found, label=label, width=bar_width, alpha=0.9)
        #plt.plot([pos[0],pos[-1]], [sum(not_found)/9, sum(not_found)/9], "--", color=f'C{i-1}', label="Insgesamt nicht gefunden")
        #plt.plot(pos, np.divide(been_found,1), ":", label="Vorhergesagte ROIs")

        i +=1

        _found_amount.append(sum(been_found))
        _not_found.append(sum(not_found))


    not_found_percent = np.multiply(np.divide(_not_found, sum(total)), 100)

    pos = np.linspace(0, (SEQS-1)*SCALES, SCALES)+SEQS/3
    axs[ax_i].plot(pos, not_found_percent, "o-", color="k", label="% nicht gefunden")

    for i in range(SCALES):
        axs[ax_i].annotate(f"{round(not_found_percent[i], 1)}", # this is the text
                 (pos[i], not_found_percent[i]), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(5,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center


    #plt.plot(np.arange(i-1)*SEQS+2.5, np.divide(_found_amount, 10), ":", marker='o', markersize=3, color='k', label="Insgesamt vorhergesagte ROIs / 10", lw=1)

    #plt.title(f"BGS - Bildskalierung")
    axs[ax_i].set_xticklabels(np.around(np.linspace(0, 1, 5), 1))
    
    axs[ax_i].set_ylim(0, 150)
    axs[ax_i].set_xlim(-1, SCALES*SEQS)

    #axs[ax_i].set_grid(axis='y', alpha=0.2, linestyle="-")

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))

    plt.legend(by_label.values(), by_label.keys(),
                loc='upper center', bbox_to_anchor=(0.5, -0.05),
                fancybox=True, shadow=True, ncol=4)

    leg = ax.get_legend()
    #leg.legendHandles[0].set_color('C7')
    #leg.legendHandles[1].set_color('C7')
    
    ax_i +=1

#plt.savefig(f"../visuals/bgs-scaling-big-iou.png", dpi=300, bbox_inches='tight')

In [ ]:
with open('../output/inference/gt_bbs_test.pickle', 'rb') as handle:
    gt_bbs = pickle.load(handle)


iou_min = [0.4, 0.2]



for rois_config_path in glob.glob(f"../output/inference/inference_scale-*.pickle"):
    config_name = os.path.basename(rois_config_path)
    
    # last param (-1), cut '.pickle', get number
    scale = float(config_name.split('_')[-1][:-7].split('-')[-1])
    
    with open(rois_config_path, 'rb') as handle:
        found_rois = pickle.load(handle)
        statistics = statistics_for_rois_pred(found_rois, gt_bbs, iou_min=iou_min, scale=scale)

        not_found = 0
        total = 0
        seqs = 0
        been_found = 0

        true_positives = 0
        false_negatives = 0
        false_positives = 0

        n_seqs = len(gt_bbs.keys())

        for seq_name, res in sorted(statistics.items()):
            not_found += res[0]
            total += res[1]
            been_found += res[2]

            true_positives += res[3]
            false_positives += res[5]
            false_negatives += res[4]
        true_negatives = been_found - true_positives - false_positives - false_negatives - not_found
        
        
        recall = true_positives / (true_positives+(false_negatives+not_found))
        precision = true_positives / (true_positives+false_positives)
        f1 = 2*recall*precision / (recall+precision)
        
        # tps, tns, fps, fns, nf
        my_colors = ["#86B300",
             "#F29718",
             "#EA6C6D",
             "#46BA94",
             "#3199E1"]
        
        plt.bar(scale, total, color='C7', width=0.1, alpha=0.4, label="Gesucht")
        
        plt.bar(scale, not_found, color=my_colors[4], label="Nicht gefunden", width=0.05)

        bottom = not_found
        plt.bar(scale, false_negatives, bottom=bottom, color=my_colors[3], label="False Negatives", width=0.05)

        bottom = np.add(bottom, false_negatives)
        plt.bar(scale, true_positives, bottom=bottom, label="True Positives", color=my_colors[0], width=0.05)

        bottom = np.add(bottom, true_positives)
        plt.bar(scale, false_positives, bottom=bottom, label="False Positives", color=my_colors[2], width=0.05)

        bottom = np.add(bottom, false_positives)
        plt.bar(scale, true_negatives, bottom=bottom, label="True Negatives", color=my_colors[1], width=0.05)
        
        TOTAL = bottom+true_negatives
        plt.plot(scale, f1*2300, "o", color="k", label=f"F1 Score")
        
        
        plt.annotate(f"{round(f1*100)}", # this is the text
                 (scale,f1*2300), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

plt.xlabel("Scale")
plt.ylim(0, 2500)
plt.xlim(0.05, 1.05)
plt.xticks(np.linspace(0.1, 1, 10))

handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(),
    loc='upper center', bbox_to_anchor=(0.5, -0.15),
    fancybox=True, shadow=True, ncol=3)


plt.show()

# Draw IoUs

In [ ]:
def draw_ious(image, gt, pred, iou):
    gt_color = (255,0,0)
    pred_color = (0, 255, 0)
    text_color = (255, 255, 0)
    linewidth = 2
    
    cv2.rectangle(image, tuple(gt[:2]), tuple(gt[2:]), gt_color, linewidth)
    cv2.rectangle(image, tuple(pred[:2]), tuple(pred[2:]), pred_color, linewidth)
    
    cv2.putText(image, "IoU: {:.4f}".format(iou), tuple(gt[:2]),
		cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 2)
    
    return image

def find_best_ious(image_dir, out_dir, gts, preds, scale):
    for seq_name in gts.keys():

        seq_dir = os.path.join(image_dir, seq_name)
        img_out_dir = os.path.join(out_dir, seq_name)
        os.makedirs(img_out_dir)

        for image_name in gts[seq_name].keys():
            image_path = f"{seq_dir}/{image_name}.jpg"
            img = imread_scaled(image_path, scale)
            
            for gt in gts[seq_name][image_name]:
                if not image_name in preds[seq_name]: break # most likely been used for BGS init
                
                gt = np.array(np.array(gt[0:4])*scale, dtype=int)
                best_iou = 0
                best_pred = [0,0,0,0]
                
                for pred in preds[seq_name][image_name]:
                    iou = bb_intersection_over_union(gt, pred)
                    
                    if iou > best_iou:
                        best_iou = iou
                        best_pred = pred[:4]
                img = draw_ious(img, gt, best_pred, best_iou)
            
            cv2.imwrite(f"{img_out_dir}/{image_name}.jpg", img)
            
            
def find_ious2(image_dir, out_dir, gts, preds, scale):
    iou_min_day, iou_night = 0.4, 0.3

    for seq_name in gts.keys():
        
        if 'day' in seq_name:
            min_iou_for_seq = iou_min_day
        else:
            min_iou_for_seq = iou_night
            
        seq_dir = os.path.join(image_dir, seq_name)
        img_out_dir = os.path.join(out_dir, seq_name)
        os.makedirs(img_out_dir)

        for image_name in gts[seq_name].keys():
            image_path = f"{seq_dir}/{image_name}.jpg"
            img = imread_scaled(image_path, scale)
            
            drawn = False
            if not image_name in preds[seq_name]: break # most likely been used for BGS init
            for pred in preds[seq_name][image_name]:
                best_iou = 0
                threshold = 0.7
                
                
                if pred[4] > threshold:
                    for gt in gts[seq_name][image_name]:
                        gt = np.array(np.array(gt[0:4])*scale, dtype=int)
                        iou = bb_intersection_over_union(gt, pred)

                        if iou > best_iou:
                            best_iou = iou

                    if best_iou < min_iou_for_seq and pred[4] > threshold:
                        img = draw_ious(img, gt, pred[:4], iou)
                        drawn = True

            if drawn:
                cv2.imwrite(f"{img_out_dir}/{image_name}.jpg", img)

In [ ]:
for rois_config_path in glob.glob(f"../output/inference/inference_scale-*.pickle"):
    config_name = os.path.basename(rois_config_path)
    
    # last param (-1), cut '.pickle', get number
    scale = float(config_name.split('_')[-1][:-7].split('-')[-1])
    print(scale)
    
    with open(rois_config_path, 'rb') as handle:
        preds = pickle.load(handle)
        
        out_dir = f"../visuals/ious/inf_scale-{scale}"
        find_ious2(c.test_sequences_dir, out_dir, gt_bbs, preds, scale)

# INFERENCE ON LOW SCALE

In [ ]:
# LOAD MODEL AND WEIGHTS
model = efn.build_model(phi=-15, dropout=0.15)
#model_checkpoints = "../output/efn/-5-dropout_0.15-weighted/checkpoints/*.hdf5"
model_checkpoints = "../output/efn_scale/-15-dropout_0.05-weighted/checkpoints/*.hdf5"
model.load_weights(max(glob.iglob(model_checkpoints), key=os.path.getctime))

img_shape = model.input_shape[1:3]

def inference(image):
    img = cv2.resize(image, img_shape) # resize incoming image to models input shape
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # cv2 reads images in BGR, model is trained on RGB
    img = np.expand_dims(img, axis=0) # model expects batches of images
    img = preprocess_input(img) # apply preprocessing function (torch-normalization)
    
    return model.predict(img)[0][0]

In [ ]:
save_roi_pred(c.test_sequences_dir,
              bmog_threshold_l=20, bmog_postprocessing_size=7,
              dilation=7, min_size=7,
              init_frames=c.frames_for_bgs_init, image_scale=0.5) # default (no scale..)



# -> rename and plot!